## 1. 환경설정

In [1]:
import torch

# CUDA 장치의 주요 버전과 부 버전을 가져옵니다.
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(8, 0)

In [2]:
%%capture
# Colab에서 torch 2.2.1을 사용하고 있으므로, 패키지 충돌을 방지하기 위해 별도로 설치해야 합니다.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # 새로운 GPU(예: Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)에 사용하세요.
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # 오래된 GPU(예: V100, Tesla T4, RTX 20xx)에 사용하세요.
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [4]:
!pip install git+https://github.com/huggingface/trl.git@82d12eb75103821cd4af1978e99b1026a90ac67d

  Cloning https://github.com/huggingface/trl.git (to revision 82d12eb75103821cd4af1978e99b1026a90ac67d) to /tmp/pip-req-build-fg7t_405
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-fg7t_405
  Running command git rev-parse -q --verify 'sha^82d12eb75103821cd4af1978e99b1026a90ac67d'
  Running command git fetch -q https://github.com/huggingface/trl.git 82d12eb75103821cd4af1978e99b1026a90ac67d
  Running command git checkout -q 82d12eb75103821cd4af1978e99b1026a90ac67d
  Resolved https://github.com/huggingface/trl.git to commit 82d12eb75103821cd4af1978e99b1026a90ac67d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.15.0.dev0-py3-none-any.whl size=316099 sha256=be33333dc6fd7182b05acdd57db76d301cb97b190fe3a114db3d2342cfdebea6
  Stored in directory: /root/.cache/pip/wheels/9a/73/31/6f0a776a79d16e581553738

In [16]:
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 105.6 MB/s eta 0:00:00
  Attempting uninstall: xformers
    Found existing installation: xformers 0.0.29.post3
    Uninstalling xformers-0.0.29.post3:
      Successfully uninstalled xformers-0.0.29.post3


## 2. 모델로드

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096  # 최대 시퀀스 길이를 설정합니다. 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
# 자동 감지를 위해 None을 사용합니다. Tesla T4, V100은 Float16, Ampere+는 Bfloat16을 사용하세요.
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용합니다. False일 수도 있습니다.
load_in_4bit = True

# 4배 빠른 다운로드와 메모리 부족 문제를 방지하기 위해 지원하는 4bit 사전 양자화 모델입니다.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Gemma 7b의 Instruct 버전
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Gemma 2b의 Instruct 버전
    "unsloth/llama-3-8b-bnb-4bit",  # Llama-3 8B
]  # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인할 수 있습니다.

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit",
    model_name="mistralai/Mistral-Small-24B-Instruct-2501",
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이를 설정합니다.
    dtype=dtype,  # 데이터 타입을 설정합니다.
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부를 설정합니다.
    # token = "hf_...", # 게이트된 모델을 사용하는 경우 토큰을 사용하세요. 예: meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.4: Fast Mistral patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # 0보다 큰 어떤 숫자도 선택 가능! 8, 16, 32, 64, 128이 권장됩니다.
    lora_alpha=32,  # LoRA 알파 값을 설정합니다.
    lora_dropout=0.05,  # 드롭아웃을 지원합니다.
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # 타겟 모듈을 지정합니다.
    bias="none",  # 바이어스를 지원합니다.
    # True 또는 "unsloth"를 사용하여 매우 긴 컨텍스트에 대해 VRAM을 30% 덜 사용하고, 2배 더 큰 배치 크기를 지원합니다.
    use_gradient_checkpointing="unsloth",
    random_state=123,  # 난수 상태를 설정합니다.
    use_rslora=False,  # 순위 안정화 LoRA를 지원합니다.
    loftq_config=None,  # LoftQ를 지원합니다.
)

Unsloth: Already have LoRA adapters! We shall skip this step.


## 3. 파일 로드

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
from google.colab import drive
import zipfile
import os
# ZIP 파일 경로 (Google Drive에 있는 파일 경로 입력)
zip_file_path = '/content/drive/MyDrive/한글리뷰/open.zip'

# 압축 해제 경로 설정
extract_to_path = '/content/extracted_files'

# 압축 해제 실행
if not os.path.exists(extract_to_path):
    os.makedirs(extract_to_path)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"ZIP 파일이 성공적으로 {extract_to_path} 경로에 압축 해제되었습니다.")

ZIP 파일이 성공적으로 /content/extracted_files 경로에 압축 해제되었습니다.


## 4. 데이터셋 생성

In [3]:
import pandas as pd
train = pd.read_csv('./extracted_files/train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('./extracted_files/test.csv', encoding = 'utf-8-sig')

In [4]:
def llama_formatting(examples):
  inputs = []
  outputs =[]
  for inp, out in zip(examples['input'], examples['output']):
    inputs.append(inp)
    outputs.append(out)

  return {"input" : inputs, "output": outputs}


llama_dataset = llama_formatting(train)

In [5]:
from datasets import Dataset
# Dataset 객체로 변환

llama_dataset_prompt_map = Dataset.from_dict(llama_dataset)
llama_dataset_prompt_map

Dataset({
    features: ['input', 'output'],
    num_rows: 11263
})

In [6]:
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN

'</s>'

In [ ]:
from datasets import load_dataset

# EOS_TOKEN은 문장의 끝을 나타내는 토큰입니다. 이 토큰을 추가해야 합니다.
EOS_TOKEN = tokenizer.eos_token

# AlpacaPrompt를 사용하여 지시사항을 포맷팅하는 함수입니다.
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""


# 주어진 예시들을 포맷팅하는 함수입니다.
def formatting_prompts_func(examples):
    instructions = examples["input"]  # 지시사항을 가져옵니다.
    outputs = examples["output"]  # 출력값을 가져옵니다.
    texts = []  # 포맷팅된 텍스트를 저장할 리스트입니다.
    for instruction, output in zip(instructions, outputs):
        # EOS_TOKEN을 추가해야 합니다. 그렇지 않으면 생성이 무한히 진행될 수 있습니다.
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,  # 포맷팅된 텍스트를 반환합니다.
    }


dataset = llama_dataset_prompt_map

# 데이터셋에 formatting_prompts_func 함수를 적용합니다. 배치 처리를 활성화합니다.
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/11263 [00:00<?, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['input', 'output', 'text'],
    num_rows: 11263
})

## 5. 모델 설정

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
tokenizer.padding_side = "right"  # 토크나이저의 패딩을 오른쪽으로 설정합니다.

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=dataset,  # 학습 데이터셋
    #eval_dataset=dataset_eval,
    dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=4096,  # 최대 시퀀스 길이
    dataset_num_proc=2,  # 데이터 처리에 사용할 프로세스 수
    packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=4,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        max_steps=3000,  # 최대 스텝 수
        do_eval=False,
        logging_steps=1,  # logging 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부, bf16이 지원되는 경우에만 사용
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=123,  # 랜덤 시드
        output_dir="outputs",  # 출력 디렉토리
    ),
)

Map (num_proc=2):   0%|          | 0/11263 [00:00<?, ? examples/s]

In [10]:
# 현재 메모리 상태를 보여주는 코드
gpu_stats = torch.cuda.get_device_properties(0)  # GPU 속성 가져오기
start_gpu_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 시작 시 예약된 GPU 메모리 계산
max_memory = round(
    gpu_stats.total_memory / 1024 / 1024 / 1024, 3
)  # GPU의 최대 메모리 계산
print(
    f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB."
)  # GPU 이름과 최대 메모리 출력
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
14.594 GB of memory reserved.


## 6. 학습

In [ ]:
trainer_stats = trainer.train()

In [12]:
# 최종 메모리 및 시간 통계를 보여줍니다.
used_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 사용된 최대 메모리를 GB 단위로 계산합니다.
used_memory_for_lora = round(
    used_memory - start_gpu_memory, 3
)  # LoRA를 위해 사용된 메모리를 GB 단위로 계산합니다.
used_percentage = round(
    used_memory / max_memory * 100, 3
)  # 최대 메모리 대비 사용된 메모리의 비율을 계산합니다.
lora_percentage = round(
    used_memory_for_lora / max_memory * 100, 3
)  # 최대 메모리 대비 LoRA를 위해 사용된 메모리의 비율을 계산합니다.
print(
    f"{trainer_stats.metrics['train_runtime']} seconds used for training."
)  # 훈련에 사용된 시간을 초 단위로 출력합니다.
print(
    # 훈련에 사용된 시간을 분 단위로 출력합니다.
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(
    f"Peak reserved memory = {used_memory} GB."
)  # 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory for training = {used_memory_for_lora} GB."
)  # 훈련을 위해 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory % of max memory = {used_percentage} %."
)  # 최대 메모리 대비 예약된 메모리의 비율을 출력합니다.
print(
    f"Peak reserved memory for training % of max memory = {lora_percentage} %."
)  # 최대 메모리 대비 훈련을 위해 예약된 메모리의 비율을 출력합니다.

3438.5856 seconds used for training.
57.31 minutes used for training.
Peak reserved memory = 21.266 GB.
Peak reserved memory for training = 6.672 GB.
Peak reserved memory % of max memory = 53.76 %.
Peak reserved memory for training % of max memory = 16.867 %.


## 7. 샘플로 추론 결과 확인해보기

In [13]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token =tokenizer.eos_token#"<|end_of_text|>"  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

In [14]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "쑴엉창 뮬 콴뤼 앉 하쉬는 트소. 엑 면쥐 퉁퉁 펄렘 듕튱 터단닉교 팝붸뀨창 않넥 되쥐 뷔륀넥갸 낱써 꾸역질잎 냐떨락교욤. 환뿡긱예 뻘뤠 둘 먀릭량 면쥐 뎡얼뤼엣 먀꼇셔 뜰얹놔톰 환긱꺄 얀 퇴교, 쳔쟝엥 꼼팡위 ㅜㅜ 확짱싫 쉴릭꼰녜돈 균텔쿤뎨 곪빵잎... 팝벡꾸 신쩡했는테 뙈짇 목살엑섯 뉼린낵캄 ㅜㅜㅜㅜ 멱딱갸 속 얀 죠햐저설 골킷 담 벗렀엇오. 솥쎄찌... 살댜 샬닥 윌렇궤 퍽뼈깍교 맏엎눈 솔쉐췻 쳐욺인넵오. 까띠 낮욘 빤쨘운 수텐 언형 쿱씩판에 낌칠랗 쿤냥 챘씩 빤쨘 죠굶쉭 떠려쳐 있섰는텟 얹쳬 뼈닮았눈짓토 묠룰 판짠.... 씰릴꼰 텁갤로 떱뻣셔 굳탯료 쥬쒸쿄  쿨뤽곱 물논뤼햐코 났왓는떼 쩌녘예 핏뿍카 웨 윕렇께 ㅜ 갼쥐럽던질. 냉붇는 구냠먈 캑꿎한테 넹쟝꼬눈 냉팡또 짤 얀 퉤교 청쑈 한낭됴 ㅠ 얀 뒈엶 잇우쉰 트소. 앝 삑윙 쌍헤오. 똔됴 야캅코 옙푸계 취엮놓흣설놓코 외 관뤼를 잎렇계 하쉰눈쥐... 놂쳔땅토 쉰쩡헷섯 타 즐끼러코 켸횐깡교 갗엊는텟 줴때롬 쎄쩍군 핫쒸눈치 읠쉼위 뒈엌 왠쥐 칩찝해썲 쿠냥 난왔눼욕. 첵꺄치 내섰긺램 염청 관뤼갸 짤됐곗견님 항꾜 귄때향꼽 갓눈뒈 따씬 얀 옭쿄 십얹오. 염긷써 멂꾼 교끼 템뮨에 ㅠ 쇽 않 좋핥적셔 닿뷴간 된칡곡길 멱코 십짖까 얀눼욜. 슈엉짱 묾 윕웨 떨탄닌는 뻘렐량 면칟갸 켸쇽 쌩꺅냐옮. 팎펙쿠쟝예 빼여 잇는 되취 냄쌔 땟문웨 잉 창솖만 뗘욜륌면 쟈똥읕롯 된찌 눌린냄갸 언샹위 됩닛댜. 쥔씸 큐역짊 낟는 켱헒 ㅠㅠ 좋궤 숟코 쉬뻤는뎨 윈쌩 콴립갸 넒뮤 엉먕친챵윌랄 칙끔토 넣뮤 휨튿려요.. 앓후..",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


max_length = model.config.max_position_embeddings  # 모델의 최대 토큰 길이
inputs["input_ids"] = inputs["input_ids"][:, :max_length]
inputs["attention_mask"] = inputs["attention_mask"][:, :max_length]


text_streamer = TextStreamer(tokenizer)
result_sentence = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=1096,  # 최대 생성 토큰 수를 설정합니다.
    use_cache=True ,
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)
import re

# ✅ 모델 응답 디코딩
decoded_output = tokenizer.batch_decode(result_sentence, skip_special_tokens=True)[0]

decoded_output


<s>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
쑴엉창 뮬 콴뤼 앉 하쉬는 트소. 엑 면쥐 퉁퉁 펄렘 듕튱 터단닉교 팝붸뀨창 않넥 되쥐 뷔륀넥갸 낱써 꾸역질잎 냐떨락교욤. 환뿡긱예 뻘뤠 둘 먀릭량 면쥐 뎡얼뤼엣 먀꼇셔 뜰얹놔톰 환긱꺄 얀 퇴교, 쳔쟝엥 꼼팡위 ㅜㅜ 확짱싫 쉴릭꼰녜돈 균텔쿤뎨 곪빵잎... 팝벡꾸 신쩡했는테 뙈짇 목살엑섯 뉼린낵캄 ㅜㅜㅜㅜ 멱딱갸 속 얀 죠햐저설 골킷 담 벗렀엇오. 솥쎄찌... 살댜 샬닥 윌렇궤 퍽뼈깍교 맏엎눈 솔쉐췻 쳐욺인넵오. 까띠 낮욘 빤쨘운 수텐 언형 쿱씩판에 낌칠랗 쿤냥 챘씩 빤쨘 죠굶쉭 떠려쳐 있섰는텟 얹쳬 뼈닮았눈짓토 묠룰 판짠.... 씰릴꼰 텁갤로 떱뻣셔 굳탯료 쥬쒸쿄  쿨뤽곱 물논뤼햐코 났왓는떼 쩌녘예 핏뿍카 웨 윕렇께 ㅜ 갼쥐럽던질. 냉붇는 구냠먈 캑꿎한테 넹쟝꼬눈 냉팡또 짤 얀 퉤교 청쑈 한낭됴 ㅠ 얀 뒈엶 잇우쉰 트소. 앝 삑윙 쌍헤오. 똔됴 야캅코 옙푸계 취엮놓흣설놓코 외 관뤼를 잎렇계 하쉰눈쥐... 놂쳔땅토 쉰쩡헷섯 타 즐끼러코 켸횐깡교 갗엊는텟 줴때롬 쎄쩍군 핫쒸눈치 읠쉼위 뒈엌 왠쥐 칩찝해썲 쿠냥 난왔눼욕. 첵꺄치 내섰긺램 염청 관뤼갸 짤됐곗견님 항꾜 귄때향꼽 갓눈뒈 따씬 얀 옭쿄 십얹오. 염긷써 멂꾼 교끼 템뮨에 ㅠ 쇽 않 좋핥적셔 닿뷴간 된칡곡길 멱코 십짖까 얀눼욜. 슈엉짱 묾 윕웨 떨탄닌는 뻘렐량 면칟갸 켸쇽 쌩꺅냐옮. 팎펙쿠쟝예 빼여 잇는 되취 냄쌔 땟문웨 잉 창솖만 뗘욜륌면 쟈똥읕롯 된찌 눌린냄갸 언샹위 됩닛댜. 쥔씸 큐역짊 낟는 켱헒 ㅠㅠ 좋궤 숟코 쉬뻤는뎨 윈쌩 콴립갸 넒뮤 엉먕친챵윌랄 칙끔토 넣뮤 휨튿려요.. 앓후..

### Response:
수영장 물 관리 안 하시는 듯요. 애 먼지 둥둥 떠다니고 바베큐장 안에 돼지 비린내가 나서 구역질이 나더라고요. 환풍기에 벌레 두 마리랑 먼지 덩어리에 마겨서 틀어놔도 환기

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n쑴엉창 뮬 콴뤼 앉 하쉬는 트소. 엑 면쥐 퉁퉁 펄렘 듕튱 터단닉교 팝붸뀨창 않넥 되쥐 뷔륀넥갸 낱써 꾸역질잎 냐떨락교욤. 환뿡긱예 뻘뤠 둘 먀릭량 면쥐 뎡얼뤼엣 먀꼇셔 뜰얹놔톰 환긱꺄 얀 퇴교, 쳔쟝엥 꼼팡위 ㅜㅜ 확짱싫 쉴릭꼰녜돈 균텔쿤뎨 곪빵잎... 팝벡꾸 신쩡했는테 뙈짇 목살엑섯 뉼린낵캄 ㅜㅜㅜㅜ 멱딱갸 속 얀 죠햐저설 골킷 담 벗렀엇오. 솥쎄찌... 살댜 샬닥 윌렇궤 퍽뼈깍교 맏엎눈 솔쉐췻 쳐욺인넵오. 까띠 낮욘 빤쨘운 수텐 언형 쿱씩판에 낌칠랗 쿤냥 챘씩 빤쨘 죠굶쉭 떠려쳐 있섰는텟 얹쳬 뼈닮았눈짓토 묠룰 판짠.... 씰릴꼰 텁갤로 떱뻣셔 굳탯료 쥬쒸쿄  쿨뤽곱 물논뤼햐코 났왓는떼 쩌녘예 핏뿍카 웨 윕렇께 ㅜ 갼쥐럽던질. 냉붇는 구냠먈 캑꿎한테 넹쟝꼬눈 냉팡또 짤 얀 퉤교 청쑈 한낭됴 ㅠ 얀 뒈엶 잇우쉰 트소. 앝 삑윙 쌍헤오. 똔됴 야캅코 옙푸계 취엮놓흣설놓코 외 관뤼를 잎렇계 하쉰눈쥐... 놂쳔땅토 쉰쩡헷섯 타 즐끼러코 켸횐깡교 갗엊는텟 줴때롬 쎄쩍군 핫쒸눈치 읠쉼위 뒈엌 왠쥐 칩찝해썲 쿠냥 난왔눼욕. 첵꺄치 내섰긺램 염청 관뤼갸 짤됐곗견님 항꾜 귄때향꼽 갓눈뒈 따씬 얀 옭쿄 십얹오. 염긷써 멂꾼 교끼 템뮨에 ㅠ 쇽 않 좋핥적셔 닿뷴간 된칡곡길 멱코 십짖까 얀눼욜. 슈엉짱 묾 윕웨 떨탄닌는 뻘렐량 면칟갸 켸쇽 쌩꺅냐옮. 팎펙쿠쟝예 빼여 잇는 되취 냄쌔 땟문웨 잉 창솖만 뗘욜륌면 쟈똥읕롯 된찌 눌린냄갸 언샹위 됩닛댜. 쥔씸 큐역짊 낟는 켱헒 ㅠㅠ 좋궤 숟코 쉬뻤는뎨 윈쌩 콴립갸 넒뮤 엉먕친챵윌랄 칙끔토 넣뮤 휨튿려요.. 앓후..\n\n### Response:\n수영장 물 관리 안 하시는 듯요. 애 먼지 둥둥 떠다니고 바베큐장 안에 돼지 비린내가 나서 구역질이 나더라고요. 환풍기에 벌레 두 마리랑 먼지 덩어리에 마겨서 틀어놔

## 8. 모델 저장

In [ ]:
# 모델 및 토크나이저 로컬 저장
output_dir = "/content/drive/MyDrive/한글리뷰/fine_tuned_model_mistralai_3000"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"모델이 로컬에 저장되었습니다: {output_dir}")


## 9. 모델 불러오기

In [ ]:
from unsloth import FastLanguageModel

output_dir = "/content/drive/MyDrive/한글리뷰/fine_tuned_model_mistralai_3000"

# FastLanguageModel로 불러오기
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=output_dir,  # 저장된 모델 경로
    max_seq_length=4096,  # 학습 시 사용한 max_seq_length과 동일하게 설정
    dtype=None,  # 학습 시 사용한 데이터 타입
    load_in_4bit=True  # 4bit 로드 여부
)

print("모델 로드 완료!")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.4: Fast Mistral patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Unsloth 2025.2.4 patched 40 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


모델 로드 완료!


## 10. 추론하기

In [22]:
from tqdm import tqdm
import re
test_result = []
FastLanguageModel.for_inference(model) # for_inference는 학습을 비활성화시켜서 추론 속도를 2배 빠르게 만듦
number = 0
for test_i in tqdm(test["input"]):
  # 모델 테스트 모듈
  number += 1
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          f"""{test_i}""", # 질문
          "",                                              # 생성된 답변칸 비어두기
      )
  ], return_tensors = "pt").to("cuda") # PyTorch 텐서, GPU사용

  outputs = model.generate(**inputs,            # 이전 입력값
                           max_new_tokens = 1096, # 최대 64개의 단어
                           stopping_criteria=stopping_criteria)    # 캐싱을 사용하여 이전 결과 저장

  t =tokenizer.batch_decode(outputs,skip_special_tokens=True)[0]
  #print(t)
  match = re.search(r'Response:\n\s*(.*)', t, re.DOTALL)
  if match:
    output_text = match.group(1).strip()
    #print(f"[0]:{output_text}")
  else:
    #print(f"[1]:{t}")
    output_text = test_i
  print(f"{number} : {output_text}")
  test_result.append(output_text)


  0%|          | 1/1689 [00:03<1:32:48,  3.30s/it]

1 : 너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이에요. 최고입니다! ㅎㅎ 다음에 또 올 것 같아요.


  0%|          | 2/1689 [00:06<1:27:32,  3.11s/it]

2 : 프론트가 없고, 조식도 없으며, 일반 입주민들이 사이사이에 있어 호텔처럼 관리가 잘 안 되는 느낌이네요. 가성비가 떨어져요.


  0%|          | 3/1689 [00:23<4:29:00,  9.57s/it]

...

62 : 너무 예쁘고 분위기가 최고예요. 처음부터 실내화 신고 들어가는 것도 깨끗하고 좋은 것 같습니다! 루프탑 가봤는데, 루프탑도 너무 좋았어요ㅠ 스파는 추워서 못했지만 다음번에 다시 오면 꼭 해보려고 합니다. 호텔원 최고입니다.


  4%|▎         | 63/1689 [07:43<2:21:18,  5.21s/it]

63 : 시내 중심에 있어 바닷가에 있는 곳보다는 저렴합니다. 겉모습도 깔끔하고 객실 내부도 깔끔하니 좋아요..


  4%|▎         | 63/1689 [07:56<3:25:10,  7.57s/it]


KeyboardInterrupt: 

## 11. 제출 csv만들기

In [18]:
submission = pd.read_csv('./extracted_files/sample_submission.csv', encoding = 'utf-8-sig')

In [19]:
submission['output'] = test_result

In [20]:
submission

,ID,output
0,TEST_0000,너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이에요. 최고입니다...
1,TEST_0001,"프론트가 없고, 조식도 없으며, 일반 입주민들이 사이사이에 있어 호텔처럼 관리가 잘..."
2,TEST_0002,진짜 불친절해요. 살면서 머물렀던 호텔 중에 최악이었습니다. 직원이지 사장인지 체크...
3,TEST_0003,뷰 맛집~~ 그런데 방음이 미흡하네요. 층간 소음과 발코니가 이중창이 아니라서 밤에...
4,TEST_0004,방 상태는 진짜 폐허 지저니데 전망은 좋아요. 보일러가 아주 잔잔하게 돌아서 추웠어...
...,...,...
1684,TEST_1684,"일반실 5만원이고, 프리미엄 6만원짜리는 스타일러스, 안마기도 있고, PC도 있지만..."
1685,TEST_1685,"일단 방은 이쁘고, 배달 음식은 금지여서 (외인지는 이문..) 룸서비스랑 조식으로 ..."
1686,TEST_1686,"웬만하면 다시 안 올 예정입니다. 거울 외풍이 있습니다. 시골. 이건 뭐, 관리할 ..."
1687,TEST_1687,도착 후 크게 기대하지 않았는데 직원분들이 친절하시고 2박 머무는 동안 매일 객실도...


In [ ]:
submission.to_csv('/content/drive/MyDrive/한글리뷰/20250214_unslothlamma한국어학습성공(mistralai)3000steps_ori.csv', index = False, encoding = 'utf-8-sig')